# Use Keras Models With Scikit-Learn For General Machine Learning
The scikit-learn library is the most popular library for general machine learning in Python.
In this lesson you will discover how you can use deep learning models from Keras with the
scikit-learn library in Python. After completing this lesson you will know:
1. How to wrap a Keras model for use with the scikit-learn machine learning library.
2. How to easily evaluate Keras models using cross validation in scikit-learn.
3. How to tune Keras model hyperparameters using grid search in scikit-learn.
Let’s get started.

## 1.1 Overview
Keras is a popular library for deep learning in Python, but the focus of the library is deep
learning. In fact it strives for minimalism, focusing on only what you need to quickly and simply
define and build deep learning models. The scikit-learn library in Python is built upon the
SciPy stack for efficient numerical computation. It is a fully featured library for general purpose
machine learning and provides many utilities that are useful in the development of deep learning
models. Not least:
1. Evaluation of models using resampling methods like k-fold cross validation.
2.  Efficient search and evaluation of model hyperparameters.

The Keras library provides a convenient wrapper for deep learning models to be used as
classification or regression estimators in scikit-learn. In the next sections we will work through
examples of using the KerasClassifier wrapper for a classification neural network created
in Keras and used in the scikit-learn library. The test problem is the Pima Indians onset of
diabetes classification dataset

### 1.2 Evaluate Models with Cross Validation
The KerasClassifier and KerasRegressor classes in Keras take an argument build fn which
is the name of the function to call to create your model. You must define a function called
whatever you like that defines your model, compiles it and returns it. In the example below
we define a function create model() that create a simple multilayer neural network for the
problem.
We pass this function name to the KerasClassifier class by the build fn argument.
We also pass in additional arguments of nb epoch=150 and batch size=10. These are auto-
matically bundled up and passed on to the fit() function which is called internally by the
KerasClassifier class. In this example we use the scikit-learn StratifiedKFold to perform
10-fold stratified cross validation. This is a resampling technique that can provide a robust
estimate of the performance of a machine learning model on unseen data. We use the scikit-learn
function cross val score() to evaluate our model using the cross validation scheme and print
the results.


In [2]:
# MLP for Pima Indians Dataset with 10-fold cross validation via sklearn
# Evaluate A Neural Network Using scikit-learn.
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
import numpy
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, init= 'uniform' , activation= 'relu' ))
    model.add(Dense(8, init= 'uniform' , activation= 'relu' ))
    model.add(Dense(1, init= 'uniform' , activation= 'sigmoid' ))
    # Compile model
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=create_model, nb_epoch=150, batch_size=10, verbose=0)
# evaluate using 10-fold cross validation
kfold = StratifiedKFold(y=Y, n_folds=10, shuffle=True, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

0.746069723253


Running the example displays the skill of the model for each epoch. A total of 10 models
are created and evaluated and the final average accuracy is displayed.

You can see that when the Keras model is wrapped that estimating model accuracy can be
greatly streamlined, compared to the manual enumeration of cross validation folds performed in
the previous lesson.

#### 1.3 Grid Search Deep Learning Model Parameters
The previous example showed how easy it is to wrap your deep learning model from Keras
and use it in functions from the scikit-learn library. In this example we go a step further. We
already know we can provide arguments to the fit() function. The function that we specify to
the build fn argument when creating the KerasClassifier wrapper can also take arguments.
We can use these arguments to further customize the construction of the model.
In this example we use a grid search to evaluate different configurations for our neural
network model and report on the combination that provides the best estimated performance.
The create model() function is defined to take two arguments optimizer and init, both of
which must have default values. This will allow us to evaluate the effect of using different
optimization algorithms and weight initialization schemes for our network. After creating our
model, we define arrays of values for the parameter we wish to search, specifically:
1. Optimizers for searching different weight values.
2. Initializers for preparing the network weights using different schemes.
3. Number of epochs for training the model for different number of exposures to the training
dataset.
4. Batches for varying the number of samples before weight updates.

The options are specified into a dictionary and passed to the configuration of the GridSearchCV
scikit-learn class. This class will evaluate a version of our neural network model for each combi-
nation of parameters (2 × 3 × 3 × 3) for the combinations of optimizers, initializations, epochs
and batches). Each combination is then evaluated using the default of 3-fold stratified cross
validation.

That is a lot of models and a lot of computation. This is not a scheme that you want to
use lightly because of the time it will take to compute. It may be useful for you to design
small experiments with a smaller subset of your data that will complete in a reasonable time.
This experiment is reasonable in this case because of the small network and the small dataset
(less than 1,000 instances and 9 attributes). Finally, the performance and combination of
configurations for the best model are displayed, followed by the performance of all combinations
of parameters.

In [ ]:
# MLP for Pima Indians Dataset with grid search via sklearn
# Grid Search Neural Network Parameters Using scikit-learn.
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.grid_search import GridSearchCV
import numpy

# Function to create model, required for KerasClassifier
def create_model(optimizer= 'rmsprop' , init= 'glorot_uniform' ):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, init= 'uniform' , activation= 'relu' ))
    model.add(Dense(8, init= 'uniform' , activation= 'relu' ))
    model.add(Dense(1, init= 'uniform' , activation= 'sigmoid' ))
    # Compile model
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform' , 'normal', 'uniform']
epochs = numpy.array([50, 100, 150])
batches = numpy.array([5, 10, 20])
param_grid = dict(optimizer=optimizers, nb_epoch=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
for params, mean_score, scores in grid_result.grid_scores_:
    print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))


Output of Grid Search Neural Network Parameters Using scikit-learn.

#### 1.4 Summary
In this lesson you discovered how you can wrap your Keras deep learning models and use them
in the scikit-learn general machine learning library. You learned:
1. Specifically how to wrap Keras models so that they can be used with the scikit-learn
machine learning library.
2. How to use a wrapped Keras model as part of evaluating model performance in scikit-learn.
3. How to perform hyperparameter tuning in scikit-learn using a wrapped Keras model.

You can see that using scikit-learn for standard machine learning operations such as model
evaluation and model hyperparameter optimization can save a lot of time over implementing
these schemes yourself.

### Next
You now know how to best integrate your Keras models into the scikit-learn machine learning
library. Now it is time to put your new skills to the test. Over the next few chapters you
will practice developing neural network models in Keras end-to-end, starting with a multiclass
classification problem next.